In [3]:
import pandas as pd
import sqlite3
import json

#cargaré los datos de los archivos JSON

customers = pd.read_json("sample_analytics.customers.json")
accounts = pd.read_json("sample_analytics.accounts.json")

#Cargar y expandir el JSON de transacciones ya que se encuentra anidado

with open("sample_analytics.transactions.json") as f:
  raw_transactions = json.load(f)

#Vamos a extraer y normalizar las transacciones por cuenta

transactions_expanded = []
for entry in raw_transactions:
    account_id = entry['account_id'] # id de cuenta que agrupa cada set de transacciones
    for tx in entry['transactions']:
        tx_flat = tx.copy()
        tx_flat['transactions_expanded'] = account_id
        if isinstance(tx['date'], dict): #con esto revisamos si la fecha esta en formato {'$date': ...}
            tx_flat['date'] = tx['date']['$date']
        transactions_expanded.append(tx_flat)

#crear dataframe de transacciones y limpiar la data
transactions = pd.DataFrame(transactions_expanded)
transactions['date'] = pd.to_datetime(transactions['date'], errors= 'coerce') #convertir a datetime
transactions = transactions[['account_id', 'date', 'amount', 'transaction_code', 'symbol', 'price', 'total']]
